# Requirements for this program to run.
* Install the following via Anaconda Prompt
    - pyqt5 (pip install pyqt5)
    - pyqt5-tools (pip install pyqt5-tools)
    - tensorflow
    - imageai
* Trianed Model and .json file (Both must be located as the same path of this file)
* Qt Designer
* Pandas

In [1]:
#Install the following via Anaconda Prompt
#-------------------------------------------------
#pip install opencv-python==4.1.2.30
#pip uninstall keras-nightly -y
#pip install keras==2.3.1
#pip install tensorflow==1.14.0
#pip install 'h5py==2.10.0' --force-reinstall
#pip install imageai==2.1.5
#pip install pyqt5
#conda install -c anaconda pandas

# Converting Image to .jpg Format

(For image detection)
This program only accepts .jpg and.png images. A python program is made named _imageConverter.py located in the test folder. To convert an image to jpg, open cmd and change the directory where the _imageConverter.py is located and type "python _imageConverter.py <image filename.format>". The converted image will be located to the same folder.

## VEHICLE DETECTION IN THERMAL IMAGES GUI

run the code below to open the GUI

In [1]:
#Importing Libraries
#-------------------------------------------------
from PyQt5 import QtWidgets, QtGui, QtCore
from PyQt5.QtWidgets import QApplication, QMainWindow, QDesktopWidget, QFileDialog, QMessageBox
from imageai.Detection.Custom import CustomObjectDetection
from imageai.Detection.Custom import CustomVideoObjectDetection
import os
import cv2
import sys
import csv 
import time
import csv
import argparse 
import datetime 
import pandas as pd
#-------------------------------------------------

#Application's Details
#-------------------------------------------------
class MyWindow(QMainWindow):
    def __init__(self):
        super(MyWindow, self).__init__()
        #Set Fixed Resolution Size to 600
        self.setFixedSize(800, 600)
        self.setWindowTitle("VEHICLE DETECTION IN THERMAL IMAGES GUI")
        #Background Color
        self.setStyleSheet("background-color: rgb(255,255,255)")
        self.initUI()
        self.center()
#-------------------------------------------------
        
#Set Form to Center Screen
#-------------------------------------------------
    def center(self):
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())
#-------------------------------------------------
    
#Code for widgets (Labels and Buttons)
#-------------------------------------------------
    def initUI(self):
            #Icon
            icon = QtGui.QIcon()
            icon.addPixmap(QtGui.QPixmap("icons/yolo_logo.png"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
            self.setWindowIcon(icon)
            self.setWindowOpacity(30.0)
            self.setToolButtonStyle(QtCore.Qt.ToolButtonTextBesideIcon)
            self.setDocumentMode(False)
            
            #Title Label
            self.title = QtWidgets.QLabel(self)
            self.title.setText("VEHICLE DETECTION IN THERMAL IMAGES USING DEEP LEARNING")    
            self.title.setFont(QtGui.QFont('Berlin Sans FB', 12))
            self.title.setStyleSheet("font-weight: bold");
            self.title.setAlignment(QtCore.Qt.AlignCenter)
            self.title.setGeometry(QtCore.QRect(0, 40, 801, 61))
            
            #Label 2
            self.label2 = QtWidgets.QLabel(self)
            self.label2.setText("Detects Vehicle in Thermal Images")    
            self.label2.setFont(QtGui.QFont('Berlin Sans FB', 12))
            self.label2.setAlignment(QtCore.Qt.AlignCenter)
            self.label2.setGeometry(QtCore.QRect(0, 90, 791, 31))       
           
            #object detection Icon
            self.iconlabel = QtWidgets.QLabel(self)
            self.iconlabel.setText("")
            self.iconlabel.setPixmap(QtGui.QPixmap("icons/tensorflow_icon.ico"))
            self.iconlabel.setScaledContents(True)
            self.iconlabel.setGeometry(QtCore.QRect(680, 30, 80, 80))
                   
            #yolov3 Label
            self.dept = QtWidgets.QLabel(self)
            self.dept.setText("TRAINED USING YOLO V3 ALGORITHM")    
            self.dept.setFont(QtGui.QFont('Berlin Sans FB', 9))
            self.dept.setScaledContents(True)
            self.dept.setWordWrap(False)
            self.dept.setGeometry(QtCore.QRect(300, 560, 801, 31))
#===================================================================            
            #Image Label
            self.Imglabel = QtWidgets.QLabel(self)
            self.Imglabel.setText("IMAGE")    
            self.Imglabel.setFont(QtGui.QFont('Berlin Sans FB', 16))
            self.Imglabel.setAlignment(QtCore.Qt.AlignCenter)
            self.Imglabel.setGeometry(QtCore.QRect(170, 190, 111, 41))
            
            #Image Button
            self.ImgBtn = QtWidgets.QPushButton(self)
            self.ImgBtn.setGeometry(QtCore.QRect(140, 230, 165, 165))
            self.ImgBtn.setText("")
            self.ImgBtn.setToolTip('Import image')
            
            #Image Icon
            imgbtnicon = QtGui.QIcon()
            imgbtnicon.addPixmap(QtGui.QPixmap("icons/image_icon.ico"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
            self.ImgBtn.setIcon(imgbtnicon)
            self.ImgBtn.setIconSize(QtCore.QSize(160, 160))
            self.ImgBtn.setFlat(True)
            
            #Image Button Click Event
            self.ImgBtn.clicked.connect(self.getImage)
#===================================================================             
            #Video Label
            self.Vidlabel = QtWidgets.QLabel(self)
            self.Vidlabel.setText("VIDEO")
            self.Vidlabel.setFont(QtGui.QFont('Berlin Sans FB', 16))
            self.Vidlabel.setAlignment(QtCore.Qt.AlignCenter)
            self.Vidlabel.setGeometry(QtCore.QRect(520, 190, 101, 51))
            
            #Video Button
            self.VidBtn = QtWidgets.QPushButton(self)
            self.VidBtn.setGeometry(QtCore.QRect(490, 230, 165, 165))
            self.VidBtn.setText("")
            self.VidBtn.setToolTip('Import video') 
            
            #Video Icon
            vidbtnicon = QtGui.QIcon()
            vidbtnicon.addPixmap(QtGui.QPixmap("icons/video_icon.ico"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
            self.VidBtn.setIcon(vidbtnicon)
            self.VidBtn.setIconSize(QtCore.QSize(160, 160))
            self.VidBtn.setFlat(True)                        
            
            #Button Click Event
            self.VidBtn.clicked.connect(self.getVideo)
#===================================================================


#Select Image from your Computer to Detect
#-------------------------------------------------
    def getImage(self):
        fname = QFileDialog.getOpenFileName(self, 'Open file', 'c:/',"Image files (*.jpg *.png)")
        #Setting Variable name "imagePath" to global so it can be called at def DetectImage()
        global imagePath
        imagePath = fname[0]
        self.DetectImage()
        
#Code For Single Image Detection
#-------------------------------------------------
    
    def DetectImage(self):
        from datetime import datetime
        classid = []
        percentage =[]
        execution_path = os.getcwd()

    # FOR STAGE CLASSIFICATION
        def forImage(output_array):
            for eachObject in output_array:
                classid.append(eachObject["name"])
                percentage.append(eachObject["percentage_probability"])
                
        detector = CustomObjectDetection()
        detector.setModelTypeAsYOLOv3()
        detector.setModelPath("detection_model-ex-014--loss-0005.977.h5")
        detector.setJsonPath("detection_config.json")
        detector.loadModel()
        detector.detectObjectsFromImage(input_image=imagePath,
                                            output_image_path=imagePath + "_detected_output_.jpg",
                                            minimum_percentage_probability=30)
        
        
        #Create CSV File from the image
        with open('single-image.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            #columns name
            writer.writerow(['CLASS', 'PERCENTAGE'])
            for (classid_x, percentage_x) in zip (classid, percentage):
                writer.writerow([classid_x, percentage_x])
        self.popupImage()
        

#Show popup Window after Finishing the Image Detection
#-------------------------------------------------
    def popupImage(self):
        msg = QMessageBox()
        msg.setWindowTitle("Image Detection")
        msg.setText("Detection Finish")
        msg.setIcon(QMessageBox.Information)
        x = msg.exec_()

#Select Video from your Computer to Detect
#-------------------------------------------------
    def getVideo(self):
        vidname = QFileDialog.getOpenFileName(self, 'Open file', 'c:/',"Video files (*.avi *.mp4)")
        #Setting Variable name "aviPath" to global so it can be called at def DetectImage()
        global aviPath
        aviPath = vidname[0]
        #call def DectectVideo
        self.DetectVideo()
        
#Code for Video 
#-------------------------------------------------
    def DetectVideo(self):
        from datetime import datetime
        classid = []
        percentage =[]
        execution_path = os.getcwd()

    # FOR VIDEO DETECTION
        def forFrame(frame_number, output_array, output_count):
            for eachObject in output_array:
                classid.append(eachObject["name"])
                percentage.append(eachObject["percentage_probability"])
                
        rec_detector = CustomVideoObjectDetection()
        rec_detector.setModelTypeAsYOLOv3()
        rec_detector.setModelPath("detection_model-ex-014--loss-0005.977.h5")
        rec_detector.setJsonPath("detection_config.json")
        rec_detector.loadModel()
        rec_detector.detectObjectsFromVideo(input_file_path=aviPath,
                                                  output_file_path=os.path.join(execution_path, aviPath + "_detected_output_"),
                                                  per_frame_function=forFrame,
                                                  minimum_percentage_probability=30,
                                                  log_progress=True) 
        
        #Create CSV File from the image
        with open('video-image.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            #columns name
            writer.writerow(['CLASS', 'PERCENTAGE'])
            for (classid_x, percentage_x) in zip (classid, percentage):
                writer.writerow([classid_x, percentage_x])
        self.popupVid()
          
#Show popup Window after Finishing the Video Detection
#-------------------------------------------------
    def popupVid(self):
        msg2 = QMessageBox()
        msg2.setWindowTitle("Video Detection")
        msg2.setText("Detection Finish")
        msg2.setIcon(QMessageBox.Information)
        y = msg2.exec_()
        
#Open GUI
if __name__ == "__main__":
    app = QApplication(sys.argv)
    win = MyWindow()
    win.show()
    sys.exit(app.exec_())

Using TensorFlow backend.
C:\Users\RYZEN\.conda\envs\myenv2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\RYZEN\.conda\envs\myenv2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\RYZEN\.conda\envs\myenv2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\RYZEN\.conda\envs\myenv2\lib\site-packages\t

Processing Frame :  1

Processing Frame :  2
Processing Frame :  3
Processing Frame :  4
Processing Frame :  5
Processing Frame :  6
Processing Frame :  7
Processing Frame :  8
Processing Frame :  9
Processing Frame :  10
Processing Frame :  11
Processing Frame :  12
Processing Frame :  13
Processing Frame :  14
Processing Frame :  15
Processing Frame :  16
Processing Frame :  17
Processing Frame :  18


C:\Users\RYZEN\.conda\envs\myenv2\lib\site-packages\imageai\Detection\Custom\__init__.py:1234: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(-x))


Processing Frame :  19
Processing Frame :  20
Processing Frame :  21
Processing Frame :  22
Processing Frame :  23
Processing Frame :  24
Processing Frame :  25
Processing Frame :  26
Processing Frame :  27
Processing Frame :  28
Processing Frame :  29
Processing Frame :  30
Processing Frame :  31
Processing Frame :  32
Processing Frame :  33
Processing Frame :  34
Processing Frame :  35
Processing Frame :  36
Processing Frame :  37
Processing Frame :  38
Processing Frame :  39
Processing Frame :  40
Processing Frame :  41
Processing Frame :  42
Processing Frame :  43
Processing Frame :  44
Processing Frame :  45
Processing Frame :  46
Processing Frame :  47
Processing Frame :  48
Processing Frame :  49
Processing Frame :  50
Processing Frame :  51
Processing Frame :  52
Processing Frame :  53
Processing Frame :  54
Processing Frame :  55
Processing Frame :  56
Processing Frame :  57
Processing Frame :  58
Processing Frame :  59
Processing Frame :  60
Processing Frame :  61
Processing 

Processing Frame :  364
Processing Frame :  365
Processing Frame :  366
Processing Frame :  367
Processing Frame :  368
Processing Frame :  369
Processing Frame :  370
Processing Frame :  371
Processing Frame :  372
Processing Frame :  373
Processing Frame :  374
Processing Frame :  375
Processing Frame :  376
Processing Frame :  377
Processing Frame :  378
Processing Frame :  379
Processing Frame :  380
Processing Frame :  381


SystemExit: 0

C:\Users\RYZEN\.conda\envs\myenv2\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
